In [ ]:
import cv2
import dlib
import tensorflow as tf
from typing import List
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import os
import cv2
from keras.models import Model
from keras.layers import SpatialDropout3D, Input, Conv3D, BatchNormalization, Activation, MaxPooling3D, Bidirectional, LSTM, Dense, TimeDistributed, ZeroPadding3D, Flatten, Dropout, GRU
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
a=""
# Load dLib's pre-trained face detector and facial landmarks predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

def load_video(path: str) -> List[float]:
    cap = cv2.VideoCapture(path)
    frames = []
    target_size = (60, 40)  # Define a fixed size for the cropped lip region (height, width)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale (dLib expects grayscale images)
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = detector(gray_frame)

        for face in faces:
            # Get the landmarks/parts for the face
            landmarks = predictor(gray_frame, face)

            # Extract the coordinates of the lips (points 48-67)
            lip_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(48, 68)]

            # Create a bounding box around the lips
            x_coords, y_coords = zip(*lip_points)
            min_x, max_x = min(x_coords), max(x_coords)
            min_y, max_y = min(y_coords), max(y_coords)

            # Crop the lip region from the frame
            lip_region = frame[min_y:max_y, min_x:max_x]

            # Resize the lip region to the target size
            lip_region_resized = cv2.resize(lip_region, target_size)

            # Convert to grayscale and append to frames
            lip_region_gray = tf.image.rgb_to_grayscale(lip_region_resized)
            frames.append(lip_region_gray)

    cap.release()

    mean = tf.math.reduce_mean(frames)
    std = tf.math.reduce_std(tf.cast(frames, tf.float32))
    return tf.cast((frames - mean), tf.float32) / std

vocab = [x for x in "abcdefghijklmnopqrstuvwxyz'?!123456789 "]
char_to_num = tf.keras.layers.StringLookup(vocabulary=vocab, oov_token="")
num_to_char = tf.keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)


def load_data(path: str):
    path = bytes.decode(path.numpy())
    frames = load_video(path)

    return frames

def mappable_function(path:str) ->List[str]:
    result = tf.py_function(load_data, [path], (tf.float32, tf.int64))
    return result

def CTCLoss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = tf.keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    return loss


inputs = Input(shape=(75, 40, 60, 1))

    # Convolutional layers with BatchNormalization and SpatialDropout3D
x = ZeroPadding3D(padding=(1, 2, 2), name='zero1')(inputs)
x = Conv3D(32, (3, 5, 5), strides=(1, 2, 2), kernel_initializer='he_normal', name='conv1')(x)
x = Activation('relu', name='actv1')(x)
x = BatchNormalization(name='batc1')(x)
x = SpatialDropout3D(0.5, name='spatial_dropout3d_1')(x)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max1')(x)
x = ZeroPadding3D(padding=(1, 2, 2), name='zero2')(x)
x = Conv3D(64, (3, 5, 5), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv2')(x)
x = Activation('relu', name='actv2')(x)
x = BatchNormalization(name='batc2')(x)
x = SpatialDropout3D(0.5, name='spatial_dropout3d_2')(x)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max2')(x)
x = ZeroPadding3D(padding=(1, 1, 1), name='zero3')(x)
x = Conv3D(96, (3, 3, 3), strides=(1, 1, 1), kernel_initializer='he_normal', name='conv3')(x)
x = Activation('relu', name='actv3')(x)    
x = BatchNormalization(name='batc3')(x)
x = SpatialDropout3D(0.5, name='spatial_dropout3d_3')(x)
x = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name='max3')(x)

    # Reshape for RNN layers
x = TimeDistributed(Reshape((-1,)), name='time_distributed_1')(x)
    # RNN layers
x = Bidirectional(GRU(256, return_sequences=True, kernel_initializer=tf.keras.initializers.Orthogonal  , name='gru1'), merge_mode='concat')(x)
x = Bidirectional(GRU(256, return_sequences=True, kernel_initializer=tf.keras.initializers.Orthogonal , name='gru2'), merge_mode='concat')(x)

    # Dense and Activation layers
x = Dense(41, kernel_initializer='he_normal', name='dense1')(x)
x = Activation('softmax', name='softmax')(x)

    # Define the model
model_lip = tf.keras.Model(inputs, x)

model_lip.load_weights("./models/dlib3_lipnet_model.h5")
model_lip.compile(optimizer=Adam(learning_rate=0.0001), loss=CTCLoss)

# Example model prediction line
def model_predict(sample):
    # Assuming your model expects input of shape (1, 75, 40, 60, 1)
    yhat = model_lip.predict(tf.expand_dims(sample, axis=0))
    return yhat

# Function to split video into chunks of (75, 40, 60, 1)
def split_video_into_chunks(video, chunk_size=75):
    """
    Splits the video into chunks of shape (75, 40, 60, 1).
    If the last chunk has fewer than 75 frames, it will be padded with zeros.
    """
    chunks = []
    total_frames = video.shape[0]  # The first dimension is the number of frames (x)

    # Iterate through the video in steps of 75 frames
    for i in range(0, total_frames, chunk_size):
        chunk = video[i:i + chunk_size]
        
        # If the chunk has fewer than 75 frames, pad it
        if chunk.shape[0] < chunk_size:
            padding = np.zeros((chunk_size - chunk.shape[0], 40, 60, 1))
            chunk = np.concatenate((chunk, padding), axis=0)
        
        chunks.append(chunk)
    
    return np.array(chunks)

# Function to process video through the model
def process_video_through_model(video):
    """
    Takes a video of shape (x, 40, 60, 1) where x is the number of frames, splits it into
    chunks of shape (75, 40, 60, 1), and runs each chunk through the model.
    """
    # Split the video into chunks of (75, 40, 60, 1)
    video_chunks = split_video_into_chunks(video)
    
    predictions = []
    
    # Run each chunk through the model
    for chunk in video_chunks:
        print(chunk.shape)
        yhat = model_lip.predict(tf.expand_dims(chunk, axis=0))
        decoded = tf.keras.backend.ctc_decode(yhat, input_length=[75], greedy=True)[0][0].numpy()
        tensor = [tf.strings.reduce_join([num_to_char(word) for word in sentence]) for sentence in decoded]
        string_value = tensor[0].numpy().decode("utf-8")
        predictions.append(string_value)
    
    # Concatenate all predictions into a single string
    full_prediction_string = ' '.join(predictions)
    
    return full_prediction_string




